In [3]:
import pandas as pd
from random import seed
seed(52)

df = pd.read_csv('prepared_dataset.csv')
df.head()

,Standard Value,smiles_encoded,exactmw,FractionCSP3,NumRings,NumAromaticRings,NumAliphaticRings,NumHeterocycles,NumAromaticHeterocycles,NumSaturatedHeterocycles,NumAliphaticHeterocycles,NumSpiroAtoms,NumBridgeheadAtoms,NumAtomStereoCenters,NumUnspecifiedAtomStereoCenters,CrippenClogP,hallKierAlpha
0,36.0,2181,513.144631,0.280000,6.0,4.0,2.0,4.0,2.0,1.0,2.0,0.0,2.0,3.0,3.0,4.8485,-3.44
1,10457.0,2181,513.144631,0.280000,6.0,4.0,2.0,4.0,2.0,1.0,2.0,0.0,2.0,3.0,3.0,4.8485,-3.44
2,40.0,2181,513.144631,0.280000,6.0,4.0,2.0,4.0,2.0,1.0,2.0,0.0,2.0,3.0,3.0,4.8485,-3.44
3,29717.0,2181,513.144631,0.280000,6.0,4.0,2.0,4.0,2.0,1.0,2.0,0.0,2.0,3.0,3.0,4.8485,-3.44
4,4.0,2205,466.108660,0.333333,5.0,3.0,2.0,4.0,2.0,1.0,2.0,0.0,2.0,3.0,3.0,4.1969,-2.79


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3041 entries, 0 to 3040
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Standard Value                   3041 non-null   float64
 1   smiles_encoded                   3041 non-null   int64  
 2   exactmw                          3041 non-null   float64
 3   FractionCSP3                     3041 non-null   float64
 4   NumRings                         3041 non-null   float64
 5   NumAromaticRings                 3041 non-null   float64
 6   NumAliphaticRings                3041 non-null   float64
 7   NumHeterocycles                  3041 non-null   float64
 8   NumAromaticHeterocycles          3041 non-null   float64
 9   NumSaturatedHeterocycles         3041 non-null   float64
 10  NumAliphaticHeterocycles         3041 non-null   float64
 11  NumSpiroAtoms                    3041 non-null   float64
 12  NumBridgeheadAtoms  

Отделим признаки от целевой переменной

In [5]:
from math import log10

df_copy = df.copy()
df['pIC50'] = df.apply(lambda x: -log10(x['Standard Value']*10**-9+10**-32), axis=1)

X = df_copy.drop(columns=['Standard Value', 'smiles_encoded'], axis=1)
Y = df['pIC50']

Выполним стандартизацию признаков.

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
x_scaler = scaler.fit_transform(X)

#pca = PCA(n_components=7)
#x_pca = pca.fit_transform(x_scaler)

Разделение выборки на тренировачную и тестовую.

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_scaler, Y, test_size=0.2, random_state=42)

Обучение классических моделей МО.

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

model_RFR = RandomForestRegressor(n_estimators=1000, max_depth=500, random_state=42)
model_RFR.fit(X_train, Y_train)

model_GBC = GradientBoostingRegressor(n_estimators=1000, max_depth=500, random_state=42)
model_GBC.fit(X_train, Y_train)

Y_predicted1 = model_RFR.predict(X_test)
Y_predicted2 = model_GBC.predict(X_test)

In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("Random Forest Regressor")
print("MAE:", mean_absolute_error(Y_test, Y_predicted1))
print("MSE:", mean_squared_error(Y_test, Y_predicted1))
print("RMSE:", (mean_squared_error(Y_test, Y_predicted1))**0.5)
print("R²:", r2_score(Y_test, Y_predicted1))

print("\nGradient Boosting")
print("MAE:", mean_absolute_error(Y_test, Y_predicted2))
print("MSE:", mean_squared_error(Y_test, Y_predicted2))
print("RMSE:", (mean_squared_error(Y_test, Y_predicted2))**0.5)
print("R²:", r2_score(Y_test, Y_predicted2))

Random Forest Regressor
MAE: 0.6698053918655738
MSE: 0.7502247115535083
RMSE: 0.8661551313439806
R²: 0.527699121729637

Gradient Boosting
MAE: 0.8015875504315894
MSE: 1.1181861435923708
RMSE: 1.0574432105755707
R²: 0.29605051719126185


Лучшая модель - Случайный лес (наименьшие показатели MAE, MSE, RMSE)